# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [5]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [8]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2025/11/11/

In [9]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [10]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'projects page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'projects page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [11]:
select_relevant_links("https://huggingface.co")

{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}

In [12]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [13]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 5 relevant links


{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [14]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 4 relevant links


{'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [15]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [16]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
Lightricks/LTX-2
Updated
6 days ago
•
861k
•
913
fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA
Updated
6 days ago
•
30.5k
•
555
tencent/HY-MT1.5-1.8B
Updated
13 days ago
•
11.7k
•
747
nvidia/nemotron-speech-streaming-en-0.6b
Updated
8 days ago
•
3.16k
•
341
LiquidAI/LFM2.5-1.2B-Instruct
Updated
5 days ago
•
15.1k
•
314
Browse 2M+ models
Spaces
Running
on
Zero
Featured
622
Qwen Image Multiple Angles 3D Camera
🎥
622
Adjust camera angles in images using 3D controls or sliders
Running
Featured
4.17k
Wan2.2 Animate
👁
4.17k
Wan2.2 Animate


In [17]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 4 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nLightricks/LTX-2\nUpdated\n6 days ago\n•\n861k\n•\n913\nfal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA\nUpdated\n6 days ago\n•\n30.5k\n•\n555\ntencent/HY-MT1.5-1.8B\nUpdated\n13 days ago\n•\n11.7k\n•\n747\nnvidia/nemotron-speech-streaming-en-0.6b\nUpdated\n8 days ago\n•\n3.16k\n•\n341\nLiquidAI/LFM2.5-1.2B-Instruct\nUpdated\n5 days ago\n•\n15.1k\n•\n314\nBrowse 2M+ models\nSpaces\nRunning\no

In [20]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [21]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 3 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is a pioneering AI and machine learning community on a mission to build the future of artificial intelligence through open collaboration. Founded in 2016 and headquartered in Paris, France, Hugging Face serves as a central hub where machine learning engineers, scientists, and enthusiasts come together to share, discover, and experiment with open-source machine learning models, datasets, and applications.

At the heart of Hugging Face’s vision is democratizing AI by fostering an open and ethical AI ecosystem, empowering the next generation of ML practitioners and end-users alike.

---

## Platform & Community

- **Machine Learning Hub:** Explore over **2 million models** and **500,000 datasets** across modalities including text, image, video, audio, and even 3D.
- **Spaces:** Interactive ML applications launched by the community, providing hands-on AI experiences—featured demos include image editing, animation, and photorealistic image generation.
- **Collaboration:** Unlimited hosting and sharing of public models, datasets, and applications encourage rapid innovation and portfolio-building for ML professionals.
- **Open Source Stack:** Accelerate development with Hugging Face’s growing ecosystem of widely-used open-source ML libraries and tools.

---

## Products & Services

- **Models:** Access and contribute to state-of-the-art natural language processing (NLP), computer vision, and other AI models.
- **Datasets:** Find curated and community-contributed datasets for training and benchmarking.
- **Spaces:** Deploy and share AI-powered web applications interactively.
- **Enterprise Solutions:** Tailored offerings with paid compute resources and enterprise-level support to empower organizations.
  
---

## Company Culture

Hugging Face champions openness, collaboration, and inclusivity with a focus on ethical AI development. The company supports an engaged, fast-growing global community aligned around transparency and knowledge-sharing.

The team includes talented engineers and scientists pushing the boundaries of AI research and development, dedicated to empowering users through cutting-edge technology combined with a friendly and accessible culture.

---

## Our Customers & Impact

- Hugging Face is widely adopted by industry leaders, academia, and individual developers worldwide who trust the platform for everything from research to production-grade AI applications.
- The community-driven approach has enabled innovations in NLP, robotics (e.g., powering expressive open-source robots), media, and other industries.
- The open platform fosters a vibrant ecosystem accelerating AI adoption and ethical standards across sectors.

---

## Careers

Join Hugging Face to be part of a passionate team committed to shaping the future of AI. The company offers exciting opportunities for:

- Machine Learning Engineers
- Research Scientists
- Software Developers
- Community Managers

Hugging Face values diversity, open collaboration, and continuous learning. Employees work in a stimulating environment where individual contributions help drive collective success in the booming AI industry.

---

## Connect With Us

Website: [huggingface.co](https://huggingface.co)  
LinkedIn: [Hugging Face LinkedIn Page](https://www.linkedin.com/company/huggingface)  
Community: Join thousands of enthusiasts on Discord, GitHub, and Twitter to collaborate and innovate.

---

## Brand Highlights

- Recognizable by its vibrant yellow (#FFD21E) and orange (#FF9D00) brand colors
- Logo and assets available in multiple formats (.svg, .png, .ai)
- Proudly part of the AI revolution with an ethical, open-source philosophy

---

Be part of the AI revolution with **Hugging Face** — **The AI community building the future.**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [23]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is a thriving AI and machine learning community dedicated to building the future through open collaboration. As one of the leading platforms in the AI space, Hugging Face empowers developers, researchers, and organizations to create, share, and discover machine learning models, datasets, and applications.

Serving as a central hub—the Hugging Face Hub—the platform fosters an open, ethical AI landscape by enabling users worldwide to contribute to and benefit from cutting-edge ML technologies.

---

## What We Offer

- **2 Million+ Models:** Browse a vast collection of public machine learning models across modalities such as text, image, video, audio, and even 3D.
- **500,000+ Datasets:** Access, contribute, and collaborate on extensive datasets powering a variety of AI solutions.
- **Spaces:** Run and share ML-powered applications and demos with easy-to-use environments.
- **Community:** Join a growing, fast-paced community of over a million users learning, sharing, and developing AI together.
- **Enterprise Solutions:** Tailored services and paid compute options that accelerate ML deployment at scale.
- **Open Source Stack:** Tools and libraries that help developers move faster and build better machine learning projects.

---

## Our Community & Culture

Hugging Face fosters a vibrant, collaborative culture aimed at democratizing AI. The platform supports a global community of machine learning engineers, data scientists, researchers, and enthusiasts who share knowledge and resources openly. The emphasis is on ethical AI development and transparency, with a strong sense of inclusion and shared purpose.

Users are encouraged to build their portfolios, share their work openly, and accelerate innovation through collective effort.

---

## Our Customers

Our users range from individual researchers and developers to major corporations and academic institutions. Hugging Face’s open platform and enterprise solutions support diverse applications including natural language processing, computer vision, audio analysis, and more — empowering innovators who need scalable, easy-to-use AI tools.

---

## Careers at Hugging Face

Join a forward-thinking company at the forefront of AI innovation! Hugging Face invites talented engineers, data scientists, researchers, and professionals passionate about open source, machine learning, and collaborative tech development to explore career opportunities.

Be part of a team dedicated to shaping the future of AI through openness, ethics, and community-driven progress.

[Explore Careers & Join Us!](https://huggingface.co/jobs)

---

## Why Choose Hugging Face?

- **Collaborative Platform:** Host, explore, and contribute to unlimited public and private ML models and datasets.
- **Cutting-Edge Tools:** Access the latest machine learning technology with an easy-to-use interface.
- **Community-Driven:** Benefit from and contribute to a vast network of AI professionals and enthusiasts.
- **Inclusive & Ethical AI:** Commitment to open, transparent, and responsible AI innovation.
- **Enterprise-Ready:** Scalable compute resources and solutions to accelerate your ML projects.

---

## Get Started Today

Sign up for free and join the leading ML community that’s shaping tomorrow’s AI.

[Sign Up Now](https://huggingface.co/join)

---

*Hugging Face – The AI community building the future.*

In [25]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the AI community building the future of machine learning. It is a dynamic collaboration platform where researchers, developers, and enthusiasts come together to create, share, and advance machine learning models, datasets, and applications. At its core, Hugging Face empowers the next generation of machine learning engineers, scientists, and end users to build open, ethical, and state-of-the-art AI technologies.

---

## What We Offer

- **2 Million+ Models:** Browse and explore a vast repository of machine learning models spanning text, image, video, audio, and even 3D modalities.
- **500,000+ Datasets:** Access and contribute to a diverse range of datasets catering to various AI applications.
- **1 Million+ Applications:** Deploy and experiment with ready-to-use AI applications built by the community.
- **Spaces:** Showcase interactive machine learning demos and apps hosted on the platform.
- **Open Source Stack:** Use the powerful Hugging Face open-source toolkit to accelerate your machine learning projects.
- **Paid Compute & Enterprise Solutions:** For scalable AI compute resources and enterprise-grade support.

---

## Community & Collaboration

Hugging Face thrives on its vibrant, fast-growing community that pushes the boundaries of AI:

- Share your ML projects and models publicly to build your professional portfolio.
- Collaborate seamlessly with thousands of AI practitioners worldwide.
- Discover trending models and datasets regularly updated by contributors and industry leaders.
- Engage in community-led discussions and contribute to ethical AI development.

---

## Company Culture

- **Open & Ethical AI:** Commitment to transparency, openness, and responsible AI innovation.
- **Innovative Environment:** Fosters creativity and continuous learning within the ML space.
- **Community-Centric:** Prioritizes collaboration, sharing knowledge, and collective growth.
- **Diverse & Inclusive:** Welcomes individuals from all backgrounds to contribute and thrive in AI.

---

## Careers & Opportunities

Joining Hugging Face means being part of the forefront of AI innovation. The company is continuously expanding and looking for talented individuals passionate about machine learning, software engineering, research, and community building.

- Work on cutting-edge open-source projects used globally.
- Collaborate with a diverse team of top researchers and engineers.
- Contribute to shaping the future of AI with ethical and impactful technology.

Check the company’s website for current openings and application details.

---

## Why Choose Hugging Face?

- The **central hub** for all things machine learning, from models to datasets to applications.
- A **robust ecosystem** supporting everything from hobbyist projects to enterprise AI solutions.
- Access to **state-of-the-art tools** and a **supportive community** that accelerates your AI development journey.
- Drive your AI projects forward with **scalable infrastructure** and **extensive resources**.
  
---

### Connect with Hugging Face

- Website: [huggingface.co](https://huggingface.co)
- Join the community and start building the future of AI today!

---

**Hugging Face – The AI community building the future.**

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>